# Matrix Completion for MDP Convolutional Codes

This notebook has some useful functions for playing around with using ideas from matrix completion to construct MDP convolutional codes. The following cells contain some methods for creating the sliding truncated generator matrices and their associated _minor product polynomials_.

In [2]:
load("convolutional_codes.sage")

In [3]:
from itertools import combinations
    
def minor_product_polynomial(G0, X=None):
    """
    Compute the value of the minor product polynomial for G0 at X
    
    If X is not specified, it is set to contain indeterminates.
    """
    
    k, n = G0.dimensions()
    G = create_generator_matrix(G0, X)
    P = 1
    
    for i in range(k + 1):
        for j in range(k, n + 1):
            if i + j != 2*k:
                continue
            
            for I in combinations(range(n), i):
                for J in combinations(range(n, 2*n), j):
                    
                    # combine I and J as a union
                    S = I + J
                    M = G[:, S]
                    P *= M.det()
                    
    return P

def minor_product_polynomial_degree(n, k):
    """Compute the degree of the minor product polynomial for any k x n MDS matrix G0"""
    
    assert 0 <= k <= n
    
    d = 0
    for i in range(k + 1):
        for j in range(k, n + 1):
                if i + j != 2*k:
                    continue
                
                d += binomial(n, i) * binomial(n, j) * (j - k)
    
    return d

def minor_product_polynomial_individual_degree(n, k):
    """
    Compute (an upper bound on) the individual degree of any variable
    in the minor product polynomial of any k x n MDS matrix G0
    """
    
    assert 0 <= k <= n
    
    d = 0
    for i in range(k):
        for j in range(k + 1, n + 1):
                if i + j != 2*k:
                    continue

                d += binomial(n, i) * binomial(n - 1, j - 1)

    return d

def minor_product_polynomial_individual_degree_standard_form(n, k):
    """
    Compute (an upper bound on) the individual degree of any variable
    in the minor product polynomial of any k x n MDS matrix G0 that is
    in standard form
    """
    
    assert 0 <= k <= n
    
    d = 0
    for i in range(k):
        for j in range(k + 1, n + 1):
                if i + j != 2*k:
                    continue

                d += binomial(n - 1, i) * binomial(n - 1, j - 1)

    return d

def print_minors(G0):
    """Compute the degree of the minor product polynomial for MDS matrix G0"""
    
    k, n = G0.dimensions()
    G = create_generator_matrix(G0)
    
    print(G)
    print()
    
    for i in range(k + 1):
        for j in range(k, n + 1):
            if i + j != 2*k:
                continue
            
            for I in combinations(range(n), i):
                for J in combinations(range(n, 2*n), j):
                    
                    S = I + J
                    M = G[:, S]
                    
                    # do sanity checks
                    assert M.det().degree() == j - k
                    assert M.det().is_homogeneous()
                    
                    print(f"{I} + {J}: {M.det()}")

def find_random_solution(G0, num_tries=10_000, seed=1):
    """
    Find a solution for X by using random sampling
    
    This will only do 10,000 tries before raising an exception.
    The random seed is set so that it produces reproducible results.
    """
    
    set_random_seed(seed)
    
    k, n = G0.dimensions()
    K = G0.base_ring()

    for _ in range(num_tries):

        X = random_matrix(K, k, (n - k))

        if satisfies_condition(G0, X):
            G = create_generator_matrix(G0, X)
            return G
    
    raise Exception(f"did not find a solution in {num_tries} tries")
    
def find_solution(G0, K=None):
    """Find a solution for X by going through all possibilities"""
    
    k, n = G0.dimensions()
    if K is None:
        K = G0.base_ring()

    for X in MatrixSpace(K, k, (n - k)):

        if satisfies_condition(G0, X):
            G = create_generator_matrix(G0, X)
            return G
    
    raise Exception(f"solution does not exist over this field")
    
def print_all_solutions(G0, K=None):
    """Print all solutions for X by going through all possibilities"""
    
    k, n = G0.dimensions()
    if K is None:
        K = G0.base_ring()

    found = False
    for X in MatrixSpace(K, k, (n - k)):

        if satisfies_condition(G0, X):
            G = create_generator_matrix(G0, X)
            found = True
            print(G)
            print()
            
    if not found:       
        print("finished without finding any solutions")

## Minor product polynomial

The next example shows how to compute the minor product polynomial for a very simple case.

In [4]:
n, k = 3, 2
K = GF(5)

G0 = vandermonde_matrix(K, k, n)
P = minor_product_polynomial(G0)

print(P)

-2*x00^2*x10 - 2*x00*x10^2 - x10^3


## Degree of minor product polynomial

With the below cell we can verify that the degree of the minor product polynomial is the same for the dual code dimension.

In [5]:
for n in range(1, 20):
    for k in range(n + 1):
        assert minor_product_polynomial_degree(n, k) == minor_product_polynomial_degree(n, n - k)
        assert minor_product_polynomial_individual_degree_standard_form(n, k) == minor_product_polynomial_individual_degree_standard_form(n, n - k)
        
print("The (individual) degree of the minor product polynomial is the same for the dual dimension")

The (individual) degree of the minor product polynomial is the same for the dual dimension


## Minors are nonzero

The following cells show that the minor polynomials are nonzero by choosing a suitable evaluation.

In [6]:
# create one of the 2k x 2k submatrices, denoted by M, and compute its determinant
n, k = 6, 3
K = GF(17)

G0 = cauchy_matrix(K, k, n)

G = create_generator_matrix(G0)
M = G[:, [0, 6, 7, 8, 9, 10]]
M.base_ring().inject_variables()
print(M)
print(M.det())

Defining x00, x01, x02, x10, x11, x12, x20, x21, x22
[  1 x00 x01 x02   0   0]
[ -8 x10 x11 x12   0   0]
[  6 x20 x21 x22   0   0]
[  0   1  -8   6  -4   7]
[  0  -8   6  -4   7   3]
[  0   6  -4   7   3   5]
-8*x01*x10 - 8*x02*x10 + 8*x00*x11 - 4*x02*x11 + 8*x00*x12 + 4*x01*x12 - 5*x01*x20 - 5*x02*x20 - 7*x11*x20 - 7*x12*x20 + 5*x00*x21 + 6*x02*x21 + 7*x10*x21 + 5*x12*x21 + 5*x00*x22 - 6*x01*x22 + 7*x10*x22 - 5*x11*x22


In [5]:
# the following evaluation shows that the polynomial M.det() is nonzero
M_eval = M.subs({x00: 1, x01: 0, x02: 0, x10: 0, x11: 1, x12: 0, x20: 0, x21: 0, x22: 0})
print(M_eval)
print(M_eval.det())

[ 1  1  0  0  0  0]
[-8  0  1  0  0  0]
[ 6  0  0  0  0  0]
[ 0  1 -8  6 -4  7]
[ 0 -8  6 -4  7  3]
[ 0  6 -4  7  3  5]
8


## Required field size

The next cells show how large the field size needs to be for some small parameters.

In [9]:
print("[n, k] : degree of polynomial -> individual degree if G0 is in standard form -> best hope")
print()

for n in range(2, 10):
    for k in range(1, n):
        print(f"[{n}, {k}] :",
              minor_product_polynomial_degree(n, k),
              "->",
              minor_product_polynomial_individual_degree_standard_form(n, k),
              "->",
              ceil(minor_product_polynomial_degree(n, k) / (k * (n - k)))
             )
    print()

[n, k] : degree of polynomial -> individual degree if G0 is in standard form -> best hope

[2, 1] : 1 -> 1 -> 1

[3, 1] : 3 -> 2 -> 2
[3, 2] : 3 -> 2 -> 2

[4, 1] : 6 -> 3 -> 2
[4, 2] : 18 -> 10 -> 5
[4, 3] : 6 -> 3 -> 2

[5, 1] : 10 -> 4 -> 3
[5, 2] : 60 -> 28 -> 10
[5, 3] : 60 -> 28 -> 10
[5, 4] : 10 -> 4 -> 3

[6, 1] : 15 -> 5 -> 3
[6, 2] : 150 -> 60 -> 19
[6, 3] : 300 -> 126 -> 34
[6, 4] : 150 -> 60 -> 19
[6, 5] : 15 -> 5 -> 3

[7, 1] : 21 -> 6 -> 4
[7, 2] : 315 -> 110 -> 32
[7, 3] : 1050 -> 396 -> 88
[7, 4] : 1050 -> 396 -> 88
[7, 5] : 315 -> 110 -> 32
[7, 6] : 21 -> 6 -> 4

[8, 1] : 28 -> 7 -> 4
[8, 2] : 588 -> 182 -> 49
[8, 3] : 2940 -> 1001 -> 196
[8, 4] : 4900 -> 1716 -> 307
[8, 5] : 2940 -> 1001 -> 196
[8, 6] : 588 -> 182 -> 49
[8, 7] : 28 -> 7 -> 4

[9, 1] : 36 -> 8 -> 5
[9, 2] : 1008 -> 280 -> 72
[9, 3] : 7056 -> 2184 -> 392
[9, 4] : 17640 -> 5720 -> 882
[9, 5] : 17640 -> 5720 -> 882
[9, 6] : 7056 -> 2184 -> 392
[9, 7] : 1008 -> 280 -> 72
[9, 8] : 36 -> 8 -> 5



In [7]:
n, k = 6, 2
q = 19
K = GF(q)

G0 = vandermonde_matrix(K, k, n).rref()

print(G0)

[ 1  0 18 17 16 15]
[ 0  1  2  3  4  5]


In [8]:
G = find_random_solution(G0, seed=None)

print(G)

Exception: did not find a solution in 10000 tries

In [ ]:
n, k = 5, 3
q = 9
K.<a> = GF(q, "a")

G0 = cauchy_matrix(K, k, n)

print(G0)

In [ ]:
G = find_solution(G0)

print(G)

In [ ]:
latex(G)

## Other ideas

In [ ]:
n, k = 5, 3
K = GF(11)

G0 = vandermonde_matrix(K, k, n).rref()

create_generator_matrix(G0)

In [ ]:
P = minor_product_polynomial(G0)

In [ ]:
print(P.degrees())

In [ ]:
best = None

for e in P.exponents():
    if best is None:
        best = e
    else:
        best = min(best, e, key=max)

print(best)

In [ ]:
P.is_squarefree()

In [ ]:
n, k = 5, 3
K = GF(11)

for _ in range(30):

    G0 = random_systematic_mds_matrix(K, k, n)
    
    P = minor_product_polynomial(G0)
    
    print(G0, len(P.exponents()))

In [ ]:
G = block_matrix([[identity_matrix(2, 2), identity_matrix(2, 2)]], subdivide=False)

In [ ]:
G

In [ ]:
n = 5
k = n - 2

assert is_prime(n)

K.<a> = GF(n^2, "a")

G0 = matrix.vandermonde(vector(K, (0..n-1))).T[:k]

x = a

X = matrix([[0, 0]]*(k - 2) + [[x, 0], [1, x]])

satisfies_condition(G0, X, verbose=True, stop=False)

In [ ]:
G = create_generator_matrix(G0, X); G

In [ ]:
M = G[:, (1, 5, 6, 7, 8, 9)]; print(M); print(M.det())

In [ ]:
minor(M, 1, 1, verbose=True)

In [ ]:
minor(M, 2, 1, verbose=True)

In [ ]:
minor(M, 2, 2, verbose=True)

In [ ]:
def minor(M, i, j, verbose=False):
    r, c = M.dimensions()
    
    rows = [0..r-1]
    rows.remove(i)
    columns = [0..c-1]
    columns.remove(j)
    
    if verbose:
        print(M[rows, columns])
    
    return M[rows, columns].det()

In [ ]:
matrix(GF(5), [[2, 4], [4, 1]])

In [ ]:
matrix(K, [[1, 1, 1], [1, 2, 4], [1, 4, 1]]).det()

In [ ]:
K.<a> = GF(4^2, "a")
K_sub = K.subfield(2, "b")
b, = K_sub.gens()

for c in K.list():
    G0 = matrix.vandermonde(vector(K, (0, 1, b, b + 1, c))).T[:3]

    X = matrix([[0, 0], [a, 0], [1, a]])
    
    if satisfies_condition(G0, X):
        print(c)

In [ ]:
help(K.subfield)

In [ ]:
K.<a> = GF(9, "a")

In [ ]:
a.minimal_polynomial()

In [ ]:
a = [1..5]
a.remove(2)

In [ ]:
a

In [ ]:
K.<a> = GF(11^2, "a")
K_sub = K.subfield(1)

R = PolynomialRing(K, 2, 2, var_array="x")
x = list(R.gens())
X = matrix([[0, 0], [x[0], 0], [x[2], x[3]]])

G0 = vandermonde_matrix(K_sub, 3, 5)

In [ ]:
P = minor_product_polynomial(G0, X)

In [ ]:
create_generator_matrix(G0, X)

In [ ]:
P.degrees()

In [ ]:
[(minor_product_polynomial_individual_degree_standard_form(n, n - 2), minor_product_polynomial_degree(n, n - 2) / (2.0*(n - 2)), n^2) for n in (4..20)]

In [ ]:
minor_product_polynomial_individual_degree_standard_form(7, 4)

In [ ]:
minor_product_polynomial_degree(7, 4) / (4*3) * 1.0